<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# xDeepFM : the eXtreme Deep Factorization Machine 
This notebook will give you a quick example of how to train an [xDeepFM model](https://arxiv.org/abs/1803.05170). 
xDeepFM \[1\] is a deep learning-based model aims at capturing both lower- and higher-order feature interactions for precise recommender systems. Thus it can learn feature interactions more effectively and manual feature engineering effort can be substantially reduced. To summarize, xDeepFM has the following key properties:
* It contains a component, named CIN, that learns feature interactions in an explicit fashion and in vector-wise level;
* It contains a traditional DNN component that learns feature interactions in an implicit fashion and in bit-wise level.
* The implementation makes this model quite configurable. We can enable different subsets of components by setting hyperparameters like `use_Linear_part`, `use_FM_part`, `use_CIN_part`, and `use_DNN_part`. For example, by enabling only the `use_Linear_part` and `use_FM_part`, we can get a classical FM model.

In this notebook, we test xDeepFM on two datasets: 1) a small synthetic dataset and 2) [Criteo dataset](http://labs.criteo.com/category/dataset)

## 0. Global Settings and Imports

In [1]:
import sys
# sys.path.append("../../")
import os
import papermill as pm
from pathlib import Path
from tempfile import TemporaryDirectory

import tensorflow as tf

from reco_utils.common.constants import SEED
from reco_utils.recommender.deeprec.deeprec_utils import (
    download_deeprec_resources, prepare_hparams
)
from reco_utils.recommender.deeprec.models.xDeepFM import XDeepFMModel
from reco_utils.recommender.deeprec.io.iterator import FFMTextIterator

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

# tmpdir = TemporaryDirectory()
tmpdir = Path('./')

System version: 3.7.9 (default, Aug 31 2020, 07:22:35) 
[Clang 10.0.0 ]
Tensorflow version: 1.15.0


In [4]:
print(tmpdir)

<TemporaryDirectory '/var/folders/r7/r_tzd68s2k59svm5wd46f7dc0000gn/T/tmphoehnwx_'>


#### Parameters

In [3]:
EPOCHS_FOR_SYNTHETIC_RUN = 15
EPOCHS_FOR_CRITEO_RUN = 50
BATCH_SIZE_SYNTHETIC = 128
BATCH_SIZE_CRITEO = 4096
RANDOM_SEED = SEED  # Set None for non-deterministic result

xDeepFM uses the FFM format as data input: `<label> <field_id>:<feature_id>:<feature_value>`  
Each line represents an instance, `<label>` is a binary value with 1 meaning positive instance and 0 meaning negative instance. 
Features are divided into fields. For example, user's gender is a field, it contains three possible values, i.e. male, female and unknown. Occupation can be another field, which contains many more possible values than the gender field. Both field index and feature index are starting from 1. <br>

## 1. Synthetic data
Now let's start with a small synthetic dataset. In this dataset, there are 10 fields, 1000 fefatures, and label is generated according to the result of a set of preset pair-wise feature interactions. 

In [4]:
data_path = tmpdir.name
yaml_file = os.path.join(data_path, r'xDeepFM.yaml')
train_file = os.path.join(data_path, r'synthetic_part_0')
valid_file = os.path.join(data_path, r'synthetic_part_1')
test_file = os.path.join(data_path, r'synthetic_part_2')
output_file = os.path.join(data_path, r'output.txt')

if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/deeprec/', data_path, 'xdeepfmresources.zip')


#### 1.1 Prepare hyper-parameters
prepare_hparams() will create a full set of hyper-parameters for model training, such as learning rate, feature number, and dropout ratio. We can put those parameters in a yaml file, or pass parameters as the function's parameters (which will overwrite yaml settings).

In [10]:
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=1000, 
                          FIELD_COUNT=10, 
                          cross_l2=0.0001, 
                          embed_l2=0.0001, 
                          learning_rate=0.001, 
                          epochs=EPOCHS_FOR_SYNTHETIC_RUN,
                          batch_size=BATCH_SIZE_SYNTHETIC)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

kg_file=None,user_clicks=None,FEATURE_COUNT=1000,FIELD_COUNT=10,data_format=ffm,PAIR_NUM=None,DNN_FIELD_NUM=None,n_user=None,n_item=None,n_user_attr=None,n_item_attr=None,iterator_type=None,SUMMARIES_DIR=None,MODEL_DIR=None,wordEmb_file=None,entityEmb_file=None,contextEmb_file=None,news_feature_file=None,user_history_file=None,use_entity=True,use_context=True,doc_size=None,history_size=None,word_size=None,entity_size=None,entity_dim=None,entity_embedding_method=None,transform=None,train_ratio=None,dim=10,layer_sizes=[100, 100],cross_layer_sizes=[1],cross_layers=None,activation=['relu', 'relu'],cross_activation=identity,user_drop

#### 1.2 Create data loader
Designate a data iterator for the model. xDeepFM uses FFMTextIterator. 

In [11]:
input_creator = FFMTextIterator

#### 1.3 Create model
When both hyper-parameters and data iterator are ready, we can create a model:

In [12]:
model = XDeepFMModel(hparams, input_creator, seed=RANDOM_SEED)

## sometimes we don't want to train a model from scratch
## then we can load a pre-trained model like this: 
#model.load_model(r'your_model_path')




Add CIN part.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Now let's see what is the model's performance at this point (without starting training):

In [13]:
print(model.run_eval(test_file))


{'auc': 0.5043, 'logloss': 0.7515}


AUC=0.5 is a state of random guess. We can see that before training, the model behaves like random guessing.

#### 1.4 Train model
Next we want to train the model on a training set, and check the performance on a validation dataset. Training the model is as simple as a function call:

In [14]:
model.fit(train_file, valid_file)

at epoch 1
train info: logloss loss:0.7556826062628493
eval info: auc:0.504, logloss:0.7042
at epoch 1 , train time: 3.4 eval time: 0.5
at epoch 2
train info: logloss loss:0.7263523223473853
eval info: auc:0.5066, logloss:0.6973
at epoch 2 , train time: 3.2 eval time: 0.6
at epoch 3
train info: logloss loss:0.7177084260380145
eval info: auc:0.5099, logloss:0.6953
at epoch 3 , train time: 3.6 eval time: 0.6
at epoch 4
train info: logloss loss:0.7118660176742527
eval info: auc:0.5147, logloss:0.6946
at epoch 4 , train time: 3.2 eval time: 0.6
at epoch 5
train info: logloss loss:0.7055103242192481
eval info: auc:0.523, logloss:0.6941
at epoch 5 , train time: 3.3 eval time: 0.6
at epoch 6
train info: logloss loss:0.6954095468078691
eval info: auc:0.5416, logloss:0.6929
at epoch 6 , train time: 3.2 eval time: 0.5
at epoch 7
train info: logloss loss:0.6723949976803101
eval info: auc:0.5916, logloss:0.6831
at epoch 7 , train time: 3.2 eval time: 0.6
at epoch 8
train info: logloss loss:0.61198

#### 1.5 Evaluate model

Again, let's see what is the model's performance now (after training):

In [16]:
res_syn = model.run_eval(test_file)
print(res_syn)
# pm.record("res_syn", res_syn)

{'auc': 0.9716, 'logloss': 0.2278}


If we want to get the full prediction scores rather than evaluation metrics, we can do this:

In [17]:
model.predict(test_file, output_file)

## 2. Criteo data 

Now we have successfully launched an experiment on a synthetic dataset. Next let's try something on a real world dataset, which is a small sample from [Criteo dataset](http://labs.criteo.com/category/dataset). Criteo dataset is a well known industry benchmarking dataset for developing CTR prediction models and it's frequently adopted as evaluation dataset by research papers. The original dataset is too large for a lightweight demo, so we sample a small portion from it as a demo dataset.

In [5]:
print('demo with Criteo dataset')
hparams = prepare_hparams(yaml_file, 
                          FEATURE_COUNT=2300000, 
                          FIELD_COUNT=39, 
                          cross_l2=0.01, 
                          embed_l2=0.01, 
                          layer_l2=0.01,
                          learning_rate=0.002, 
                          batch_size=BATCH_SIZE_CRITEO, 
                          epochs=EPOCHS_FOR_CRITEO_RUN, 
                          cross_layer_sizes=[20, 10], 
                          init_value=0.1, 
                          layer_sizes=[20,20],
                          use_Linear_part=True, 
                          use_CIN_part=True, 
                          use_DNN_part=True)

train_file = os.path.join(data_path, r'cretio_tiny_train')
valid_file = os.path.join(data_path, r'cretio_tiny_valid')
test_file = os.path.join(data_path, r'cretio_tiny_test')

demo with Criteo dataset
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
model = XDeepFMModel(hparams, FFMTextIterator, seed=RANDOM_SEED)

# check the predictive performance before the model is trained
print(model.run_eval(test_file)) 
model.fit(train_file, valid_file)
# check the predictive performance after the model is trained
res_real = model.run_eval(test_file)
print(res_real)
# pm.record("res_real", res_real)

Add linear part.
Add CIN part.
Add DNN part.
{'auc': 0.4728, 'logloss': 0.7113}
at epoch 1
train info: logloss loss:744.360179901123
eval info: auc:0.6637, logloss:0.5342
at epoch 1 , train time: 32.2 eval time: 4.7
at epoch 2
train info: logloss loss:385.6692409515381
eval info: auc:0.7137, logloss:0.5109
at epoch 2 , train time: 31.8 eval time: 5.2
at epoch 3
train info: logloss loss:191.50827884674072
eval info: auc:0.7283, logloss:0.5037
at epoch 3 , train time: 32.4 eval time: 5.1
at epoch 4
train info: logloss loss:92.20772361755371
eval info: auc:0.7359, logloss:0.4991
at epoch 4 , train time: 31.9 eval time: 5.2
at epoch 5
train info: logloss loss:43.1594443321228
eval info: auc:0.74, logloss:0.4963
at epoch 5 , train time: 31.6 eval time: 4.9
at epoch 6
train info: logloss loss:19.656914949417114
eval info: auc:0.7426, logloss:0.4946
at epoch 6 , train time: 33.0 eval time: 4.8
at epoch 7
train info: logloss loss:8.770353466272354
eval info: auc:0.7441, logloss:0.4934
at epoch

## Reference
\[1\] Lian, J., Zhou, X., Zhang, F., Chen, Z., Xie, X., & Sun, G. (2018). xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems. Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery \& Data Mining, KDD 2018, London, UK, August 19-23, 2018.<br>

In [13]:
# Cleanup
# tmpdir.cleanup()